In [1]:
import pandas as pd
import nltk
from itertools import chain
import pycrfsuite
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import sklearn
from nltk import word_tokenize
import pdb
from collections import OrderedDict
from sklearn.externals import joblib

In [2]:
filename ="Hotels_test_data.xlsx"
df = pd.read_excel(filename,"TrainingData")
classes = (pd.read_excel(filename, "Classes"))['classes'].tolist()

In [3]:
def split_reviews(sent):
    try:
        splitted_review = sent['content_and_phrases'].split("|")
        review = splitted_review[0]
        phrases = splitted_review[1:]
        themes = sent['gl_themes'].split("|")
        if len(themes)!=len(phrases):
            return 0
        else:
            return review, phrases, themes
    except:
        pdb.set_trace()

In [4]:
input_ = (df[['content_and_phrases','gl_themes']].apply(split_reviews,axis=1)).tolist()

In [5]:
for i in xrange(len(input_)):
    while True:
        try:
            input_[i][1].remove("")
            input_[i][2].remove("")
        except ValueError:
            break

In [6]:
def contains(small, big):
    for i in xrange(len(big)-len(small)+1):
        for j in xrange(len(small)):
            if big[i+j] != small[j]:
                break
        else:
            return i, i+len(small)
    return False

In [7]:

count = 0
# contains(word_tokenize(input_[0][1][2]),word_tokenize(input_[0][0]))
def filter_eos(a):
    a = filter(lambda a: a != "," and a != "." and a != "?" and a != "!", a)
    return a

In [8]:
def BIOtagging_sentence(row_, count):
    review = row_[0]
    themes = row_[2]
    phrases = row_[1]
    tokenized_review = filter_eos(word_tokenize(review))
    pos_tags = nltk.pos_tag(tokenized_review)
    BIO_tags = []
    for i in xrange(len(tokenized_review)):
        BIO_tags.append((tokenized_review[i],pos_tags[i][1],"O"))
    for j in xrange(len(phrases)):
        phrase_tokens = filter_eos(word_tokenize(phrases[j]))
        try:
            class_theme = classes.index(themes[j])
        except:
            continue
        if contains(phrase_tokens, tokenized_review) != False:
            start, end = contains(phrase_tokens, tokenized_review)
            BIO_tags[start] = (tokenized_review[start],pos_tags[start][1],"B-"+str(class_theme))
            for k in xrange(start+1, end):
                BIO_tags[k] = (tokenized_review[k],pos_tags[k][1],"I-"+str(class_theme))
        else:
            count = count + 1
    return BIO_tags, count
    

In [9]:
all_tags = []
count = 0
for i in xrange(len(input_)):
    tags_, count = (BIOtagging_sentence(input_[i], count))
#     count = count + count_
    all_tags.append(tags_)

In [10]:
joblib.dump(all_tags, "Trained NER Models/test_BIO_Hotels.pkl")

['Trained NER Models/test_BIO_Hotels.pkl']